# Project Outline

## Introduction

Molecular dynamics simulations are a method that applies equations of motion for each specified particle in a defined system to directly simulate the dynamics of the particles within a defined system. Molecular dynamics in the field of chemistry has allowed scientists to observe the dynamics of a system in microscopic detail which is usually not possible with traditional methods of chemical analysis. The use of MD has been applied to a variety of chemical reactions and molecules. It is used to model chemical processes and even used as an alternative method to carry out chemical reactions. This has allowed a great reduction in the labor, waste, and resources used in traditional lab settings. 

This project will take advantage of this method to simulate molecules of water in a system that is spherically shaped. Once the system is set up, we will simulate the system heating up and observe how that affects the system. We will simulate it so that the molecules go through all 3 of its phases. We will be comparing the density as a function of radius and numerically taking the derivative to investigate how density changes with respect to radius. 


## Numerical Method
A numerical derivative will be implemented to see how density changes as a with respect to radius. The numerical derivative will implement the standard definition of the derivative (see below) with some alterations to improve accuracy. From the standard definition, we cannot take the limit (h-->0). 
$$\frac{\mathrm{df} }{\mathrm{d} x}=\lim_{h \to 0 }\frac{f(x+h)-f(x)}{h}$$

However, by making the size of h small, an approximation can be calculated. Accuracy can be further improved by using the central difference (see below). This alteration to the definition of the derivative still approximates the derivative using the difference between two values of f(x) at points a distance h apart. The only thing that has changed is that the two points are now placed symmetrically around point x. One point is at 0.5h above x and the other is 0.5h below x. 
$$
\frac{\mathrm{df} }{\mathrm{d} x}\approx\frac{f(x+h/2)-f(x-h/2)}{h}
$$

Since we are taking the derivative of a sampled set of data, we do not know the value of the function at x+0.5h or x-0.5h. Thus, the equation must be altered further to fit this situation. By using points in the sample that are 2h apart the central difference method can be applied. The formula for the central difference will now become:
$$
\frac{\mathrm{df} }{\mathrm{d} x}\approx\frac{f(x+h)-f(x-h)}{2h}
$$


## Preliminary Results from Simulation Analysis

Once the simulations were complete, they were visualized using VMD software. Visually, it is apparent that the as temperature increases the motion of the molecules also increase. This is consistent with what is expected. As temperature increases, thermal energy within the system also increases and this allows molecules to reach higher energy states and have more kinetic energy. It is difficult to qualitatively compare density because measuring tools were not accessed using VMD. However, because the molecules are moving as theoretically expected, this gives a good sign that the density of water will behave as expected as well.  

The videos are available to watch in the folder called Videos in project repsository

Raw data can be accesed [here.](https://pacificu.box.com/v/WaterSimulationData)

Code for running the simulations is in the file called WaterSimulationCode.ipynb in project repository.

#  Etracting values from simulation results

In [69]:
# will take each pdb file and parse it

def file_extract(file):
    """
    Takes a file and extracts the atom type, atom number,
    xposition, ypostion, and zposition and puts them into seperate lists.
    Note that each frame is not seperated yet. 
    
    Inputs:
    file- pdb file, assumed that file is in the same directory
    
    Returns: atomtype, atomnumber, xposition, yposition, zpositions in lists
    
    Example:
    >>> file_extract("example.pdb")
    (['el10'], ['el1'], ['el6'], ['el7'], ['el8'])
    """
    
    file= open(file)

    #creates empty lists for the sparsed file to go into
    atomtype=[]
    atomnumber=[]
    xposition=[]
    yposition=[]
    zposition=[]
    
    for line in file:
        splitline = line.split() #splits each line of file so each goup of text is a string basically omits spaces
        if splitline[0] == "HETATM":
            atomtype.append(splitline[-1])
            atomnumber.append(splitline[1])
            xposition.append(splitline[6])
            yposition.append(splitline[7])
            zposition.append(splitline[8])

    return atomtype, atomnumber, xposition, yposition, zposition  


  
avgxposition=[]
for atom in range(6000):
    if atomnumber == atom:
        templist=[]
        templist.append(xposition[atom])

 


In [73]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=1)